# Read Data

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# for LR
import statsmodels.api as sm
from sklearn.model_selection import KFold
import random

# for logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# for Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# for feature eng
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [ ]:
%%time
train_data = pd.read_csv('/Users/josephcolaco/customer_revenue_prediction/data/train.csv')
test_data = pd.read_csv('/Users/josephcolaco/customer_revenue_prediction/data/test.csv')
print('Shape of test data is', test_data.shape)
print('Shape of train data is',train_data.shape)

# Data Cleaning

To do for data cleaning
- json to dataframe columns
- format various data types
- need to format the date field
- visitNumber as int
- format visitStartTime
- format visits
- format campaignCode
- deal with nulls
- drop columns that are not needed

### Convert json columns to df format

In [ ]:
%%time
def json_to_lst(model_data, json_cols):
    for column in json_cols:
        model_data = model_data.join(pd.DataFrame(
            model_data.pop(column).apply(pd.io.json.loads).values.tolist(), index=model_data.index)) 
    
    return model_data

In [ ]:
%%time
json_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']
train_data_v1 = json_to_lst(train_data, json_cols)
test_data_v1 = json_to_lst(test_data, json_cols)

The 'adwordsClickInfo' feild is a dictionary. Let's change that to a dataframe columns.

In [ ]:
%%time
concat_df = pd.DataFrame(train_data_v1['adwordsClickInfo'].tolist())
train_data_v2 = train_data_v1.drop(columns = ['adwordsClickInfo'])
train_data_v3 = pd.concat([train_data_v2,concat_df], axis=1)

### Drop Columns with No Data

In [ ]:
print('Shape of train data version 3 is:', train_data_v3.shape)
train_data_v3.head(3)

Data is not available in several columns in the dataset. These columns will be dropped from the dataframe. 'visits' field will also be deleted as it has only one value in the column ('1').

In [ ]:
train_data_v4 = train_data_v3.drop(columns = ['browserSize', 'browserVersion', 'flashVersion', 'language','mobileDeviceBranding', 
                             'mobileDeviceInfo', 'mobileDeviceMarketingName', 'mobileDeviceModel', 
                              'mobileInputSelector', 'operatingSystemVersion', 'screenColors', 'screenResolution',
                             'cityId', 'latitude', 'longitude', 'networkLocation', 'targetingCriteria', 
                                              'criteriaParameters', 'visits', 'socialEngagementType', 'isTrueDirect'])

print('Shape of train_data version 4 is:', train_data_v4.shape)

### Adjusting Data Types

#### Proper Formating for Data Fields:
- date to datetime
- hits to int
- newVisits to int
- pageviews to int
- transactionRevenue to float64
- page to int
- bounces to int
- visitStartime to datetime

In [ ]:
train_data_v4['date'] = pd.to_datetime(train_data_v4['date'], format='%Y%m%d')
train_data_v4['transactionRevenue'] = train_data_v4['transactionRevenue'].fillna(1).astype('float64')
train_data_v4['hits'] = train_data_v4['hits'].fillna(0).astype('int64')
train_data_v4['newVisits'] = train_data_v4['newVisits'].fillna(0).astype('int64')
train_data_v4['pageviews'] = train_data_v4['pageviews'].fillna(0).astype('int64')
train_data_v4['page'] = train_data_v4['page'].fillna(0).astype('int64')
train_data_v4['bounces'] = train_data_v4['bounces'].fillna(0).astype('int64')
train_data_v4['isVideoAd'] = train_data_v4['isVideoAd'].fillna('False').astype('bool')
train_data_v4['visitStartTime'] = train_data_v4['visitStartTime'].apply(lambda x: dt.datetime.utcfromtimestamp(float(x)))
train_data_v4['visitHour'] = train_data_v4['visitStartTime'].dt.hour

### Deal with Nulls

In [ ]:
null_df = pd.DataFrame(train_data_v4.isnull().sum(), columns = ['count_null_rows'])
# null_df = null_df[null_df['count_null_rows']>0]
null_df = null_df[null_df['count_null_rows']>0].sort_values(by='count_null_rows', ascending=False)
null_df

fill all object column nans with '(not set)'

In [ ]:
for i in train_data_v4.columns:
    if train_data_v4[i].isnull().sum() > 0:
        train_data_v4[i] = train_data_v4[i].fillna('(not set)')

Replace all 'not available in demo dataset' to '(not set)'

In [ ]:
train_data_v4 = train_data_v4.replace(['not available in demo dataset', '(not set)'], ['(not_set)', '(not_set)'])

In [ ]:
train_data_v4.isnull().sum().sum()

### Log Transaction Revenue

In [ ]:
train_data_v4 = train_data_v4.assign(
    logtransactionRevenue = np.log(train_data_v4.transactionRevenue))
train_data_v4 = train_data_v4.drop(columns=['transactionRevenue'])

### Make Binary Made Purchase Columns

In [ ]:
train_data_v4['madePurchase'] = np.where(train_data_v4['logtransactionRevenue'] > 0, 1, 0)

### Understanding the Unique Identifier

In [ ]:
train_data_v4.head()
print('shape of train data version 4:', train_data_v4.shape[0])
print('number of unique session_ids is:', train_data_v4.sessionId.nunique())
print('diff is:', train_data_v4.shape[0]-train_data_v4.sessionId.nunique())
print('The sessionId should be a unique identifier. Investigate duplicates:')

The max count of duplicate sessionId is 2. This method does the following: 
    - if train data
        - Delete first of two duplicate rows as long as the first row does not have a transaction revenue > 0
    - if test_data
        - delete the first row of duplicate sessionId
Only 1 row in the train_data has a transacton revenue > 0 removed and thats because bith duplicate sessions had a revenue

In [ ]:
def remove_session_id_dup(data, is_train):
    sessionId_counts = pd.DataFrame(data['sessionId'].value_counts()).reset_index()
    dup_sessionId = sessionId_counts['index'][sessionId_counts['sessionId']>1].values.tolist()
    dup_data = data[data['sessionId'].isin(dup_sessionId)].sort_values(by='sessionId')
    
    index_to_delete = []
    count = 0
    delete = False
    for index, row in dup_data.iterrows():
        if is_train:
            if count == 0 and row['madePurchase'] == 1: #leaves the first duplicate if the person has made a purchase
                count +=1
                delete = False
                continue
            elif count == 0 and delete == False: #deletes the first duplicate if person hasnt made a purchase
                index_to_delete.append(index)
                delete = True
                count +=1
            elif count == 1 and delete == False: #deletes the second duplicate if person has made a purchas in the first row
                index_to_delete.append(index)
                count = 0
            elif count == 1 and delete == True: #assign delete to false and continues
                delete = False
                count = 0
                continue
        elif count == 0 and delete == False:
            index_to_delete.append(index)
            delete = True
            count +=1
        else:
            delete = False
            count = 0
            
    data = data.reset_index()
    print('Deleted', len(index_to_delete), 'rows!')
    print('Shape of data before deleting duplicates:', data.shape)
    data_v1 = data[~data['index'].isin(index_to_delete)]
    print('Shape of data after deleting duplicates:', data_v1.shape)   

    return index_to_delete, data_v1

In [ ]:
train_index_delete, train_data_v5 = remove_session_id_dup(train_data_v4, True)
test_index_delete, test_data_v1 = remove_session_id_dup(test_data, False)

# Feature Engineering

### Make Day, Month and Year Columns from Date

In [ ]:
train_data_v5['dayNameDate'] = train_data_v5['date'].dt.day_name()
train_data_v5['monthDate'] = train_data_v5['date'].dt.month
train_data_v5['yearDate'] = train_data_v5['date'].dt.year
train_data_v5['dayDate'] = train_data_v5['date'].dt.day

In [ ]:
train_data_v5.head()

# EDA

### EDA Methods

In [ ]:
def plot_hist(x, bins, xlabel, ylabel, title):
#     plt.style.use('ggplot')
    plt.style.use('seaborn')
    plt.hist(x, bins)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

In [ ]:
def plot_scatter(x, y,xlabel, ylabel, title):
    plt.style.use('seaborn')
    plt.scatter(x, y, alpha=0.5)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

In [ ]:
def plot_sns_scatter(data, x, y, hue):
    sns.lmplot(x=x, y=y, data=data, 
               fit_reg=False, legend=False, hue = hue)

In [ ]:
def plot_cat_bar(data, col, title):
    plt.style.use('seaborn')
    col_value_counts = pd.DataFrame(data[col].value_counts())
    if col_value_counts.shape[0] > 10:
        col_value_counts = col_value_counts[:10]
    plt.bar(col_value_counts.index, col_value_counts[col])
    plt.title(title)
    plt.show()

## Analysis on Record Level

### Analysis on how many records made a purchase

In [ ]:
col_value_counts = pd.DataFrame(train_data_v5['madePurchase'].value_counts())
print('Percentage of records that made a transaction:', 
      round(train_data_v5[train_data_v5['madePurchase'] ==1].shape[0]/train_data_v5.shape[0], 2))
col_value_counts

In [ ]:
sns.countplot(x=train_data_v5['madePurchase'])

In [ ]:
train_data_v5.groupby('madePurchase').mean()

You can tell clearly from above that the records that made a purchase are ones that have high pageviews, hits, 0 bounces and often not from a mobile device. The transactions are generally later in the day as well

In [ ]:
train_data_v5.groupby('deviceCategory').mean()
# a major proportion of the logtransactionrevenue is from desktop.

In [ ]:
train_data_v4.groupby('channelGrouping').mean()

In [ ]:
train_data_v4.groupby('continent').mean()

In [ ]:
cross_tab_df = pd.crosstab(train_data_v5.deviceCategory, train_data_v5.madePurchase)
cross_tab_df

In [ ]:
cross_tab_df = pd.crosstab(train_data_v5.operatingSystem, train_data_v5.madePurchase)
cross_tab_df

In [ ]:
cross_tab_df = pd.crosstab(train_data_v5.isMobile, train_data_v5.madePurchase)
cross_tab_df

In [ ]:
cross_tab_df = pd.crosstab(train_data_v5.browser, train_data_v5.madePurchase)
cross_tab_df

In [ ]:
cross_tab_df = pd.crosstab(train_data_v5.continent, train_data_v5.madePurchase)
cross_tab_df

In [ ]:
cross_tab_df = pd.crosstab(train_data_v5.metro, train_data_v5.madePurchase)
cross_tab_df

In [ ]:
cross_tab_df = pd.crosstab(train_data_v5.monthDate, train_data_v5.madePurchase)
cross_tab_df

In [ ]:
cross_tab_df = pd.crosstab(train_data_v5.dayDate, train_data_v5.madePurchase)
cross_tab_df

In [ ]:
col_value_counts = pd.DataFrame(train_data_v5['networkDomain'].value_counts())
# print('Percentage of records that made a transaction:', 
#       round(train_data_v5[train_data_v5['madePurchase'] ==1].shape[0]/train_data_v5.shape[0], 2))
col_value_counts[:10]

In [ ]:
object_cols = train_data_v4.select_dtypes(include='object').columns
for i in object_cols:
    if i != 'fullVisitorId' and i != 'sessionId':
        if train_data_v4[i].nunique() <= 50:
            print(i)
            plot_cat_bar(train_data_v4, i, str(i) + ' vs frequency')
# plt.bar(test.index, test.channelGrouping)

In [ ]:
plot_hist(train_data_v5['logtransactionRevenue'][train_data_v5['logtransactionRevenue'] > 0], 10, 
          'Log TransactionRevenue', 'Frequency', 'Hist of Log TransactionRevenue')

In [ ]:
numeric_cols = train_data_v4.select_dtypes(include='number').columns
for i in numeric_cols:
    if i != 'visitId':
        plot_hist(train_data_v4[i], 10, 
          str(i), 'Frequency', 'Hist of ' + str(i))

Ideas for Modeling
1) Delete all records that contain a web browser with not purchases. Train a model to classify madepurchase and then train another model to predict the logtransaction revenue

In [ ]:
plot_hist(train_data_v4['visitNumber'][train_data_v4['visitNumber'] < 50], 10, 
          'visitNumber', 'Frequency', 'Hist of visitNumber')

In [ ]:
plot_hist(train_data_v4['hits'][train_data_v4['hits'] < 50], 10, 
          'hits', 'Frequency', 'Hist of hits')

In [ ]:
plot_hist(train_data_v4['pageviews'][train_data_v4['pageviews'] < 50], 10, 
          'pageviews', 'Frequency', 'Hist of pageviews')

In [ ]:
numeric_data = train_data_v4.select_dtypes(include='number')
corr_LTR = pd.DataFrame(
    numeric_data.corrwith(numeric_data['logtransactionRevenue'], axis=0, drop=False), columns = ['corr'])
corr_LTR.sort_values(by='corr', ascending=False)

In [ ]:
for i in numeric_cols:
    if i != 'visitId':
        plot_sns_scatter(train_data_v4, i, 'logtransactionRevenue', 'madePurchase')

## Feature Engineering on Columns with High Number of Unique Values

In [ ]:
n_unique = pd.DataFrame(train_data_v5.nunique(), columns=['count_unique'])
n_unique = n_unique.sort_values(by='count_unique', ascending=False)
print('These are all the columns and the number of unique values in them:')
n_unique

### Focusing in on networkDomain

In [ ]:
train_data_v5['networkDomain'] = train_data_v5['networkDomain'].replace(['unknown.unknown'], ['(not_set)'])

In [ ]:
networkDomain = train_data_v5[['networkDomain','madePurchase']]

In [ ]:
networkDomain['networkDomain'].value_counts()

In [ ]:
networkDomain['splitNetworkDomain'] = networkDomain['networkDomain'].str.split('.')

In [ ]:
networkDomain['topLevelDomain'] = networkDomain['networkDomain'].str.split('.').str[-1]

In [ ]:
print('There are now', networkDomain['topLevelDomain'].nunique(), 'unique values for top level domain')

In [ ]:
print('Here are the value counts:')
networkDomain['topLevelDomain'].value_counts()

In [ ]:
network_crosstab = pd.crosstab(networkDomain.topLevelDomain, networkDomain.madePurchase)
network_crosstab.sort_values(network_crosstab.columns[1], ascending = False)

In [ ]:
def pop_network_domain(data):
    top_level_domain = ['(not_set)', '.us', '.net', '.com', '.edu', '.ca', '.org', '.mx' ]
    data_v1 = data
    for i in top_level_domain:
        col_name = 'tl_' + str(i)
        data_v1[col_name] = data_v1['networkDomain'].str.contains(i, case=1, na=0)
    return data_v1

In [ ]:
networkDomain_v1 = pop_network_domain(networkDomain)
networkDomain_v1.head()

In [ ]:
train_data_v6 = pop_network_domain(train_data_v5)

In [ ]:
train_data_v6.head()

In [ ]:
train_data_v5.shape

### Focusing in on gclID

In [ ]:
train_data_v6['gclId'].value_counts()

In [ ]:
crosstab = pd.crosstab(train_data_v6.gclId, train_data_v6.madePurchase)
crosstab.sort_values(crosstab.columns[1], ascending = False)

Because of my lack of understanding of gclId I will not use this in the model

### Focusing on keyword

In [ ]:
train_data_v6['keyword'] = train_data_v6['keyword'].replace(['(not provided)'], ['(not_set)'])

In [ ]:
train_data_v6['keyword'].nunique()

In [ ]:
crosstab = pd.crosstab(train_data_v6.keyword, train_data_v6.madePurchase)
crosstab.sort_values(crosstab.columns[1], ascending = False)

In [ ]:
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
def stem_sentences(sentence):
    sentence = re.sub('[^a-zA-Z]+', ' ', sentence)
    tokens = sentence.split()
    filtered_sentence = [w for w in tokens if not w in stop_words] 
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens if len(token) > 1]
    return ' '.join(stemmed_tokens)

In [ ]:
%%time
porter_stemmer = PorterStemmer()
train_data_v6['cleanKeyword_v2'] = train_data_v6['keyword'].apply(stem_sentences)

In [ ]:
train_data_v6['cleanKeyword'].nunique()
train_data_v6['cleanKeyword_v2'].nunique()

In [ ]:
crosstab = pd.crosstab(train_data_v6.cleanKeyword_v2, train_data_v6.madePurchase)
crosstab.sort_values(crosstab.columns[1], ascending = False)

In [ ]:
keyword_w_purchase = crosstab.sort_values(crosstab.columns[1], ascending = False)[1:7].index

In [ ]:
keyword_w_purchase

In [ ]:
lst_words = []
for i in keyword_w_purchase:
    for j in i.split(' '):
        if len(j) > 2:
            lst_words.append(j)
lst_words      

In [ ]:
lst_words_df = pd.DataFrame(lst_words, columns=['words'])

In [ ]:
lst_words_df_v2 = pd.DataFrame(lst_words_df['words'].unique(), columns=['words'])

In [ ]:
print(lst_words_df.shape)
print(lst_words_df_v2.shape)

In [ ]:
lst_words_final = lst_words_df_v2['words'].values
lst_words_final

In [ ]:
def transform_keyword(data):
    # given the data, want to make binary columns stating that they contain necessary keywords 
    lst_words_final = ['qehscssdk', 'googl', 'merchandis', 'store', 'hzbaqlcbjwfgoh', 
                       'remarket', 'content', 'target', 'zknv']
    data_v1 = data
    data_v1['clean_keyword'] = data_v1['keyword'].apply(stem_sentences)
    for i in lst_words_final:
        col_name = 'keyword_' + str(i)
        data_v1[col_name] = data_v1['clean_keyword'].str.contains(i, case=1, na=0)
        
    return data_v1

In [ ]:
%%time
train_data_v7 = transform_keyword(train_data_v6)

### Focusing in on referralPath

In [ ]:
train_data_v7['referralPath'] = train_data_v7['referralPath'].replace(['/'], ['(not_set)'])

In [ ]:
pd.DataFrame(train_data_v7['referralPath'].unique())

In [ ]:
crosstab = pd.crosstab(train_data_v6.referralPath, train_data_v6.madePurchase)
crosstab = crosstab.sort_values(crosstab.columns[1], ascending = False)
crosstab[2] = crosstab[1]/crosstab[0]
crosstab = crosstab.sort_values(crosstab.columns[2], ascending = False)
crosstab

In [ ]:
filterd_crosstab = crosstab[(crosstab[1] > 5) & (crosstab[2] > 0.01) & (crosstab.index != "(not_set)")]
filterd_crosstab

In [ ]:
filterd_crosstab_index = pd.DataFrame(filterd_crosstab.index)
filterd_crosstab_index

In [ ]:
a = 'hela  ssd  '
a.strip()

In [ ]:
def stem_sentences_v2(sentence):
    sentence = re.sub('[^a-zA-Z]+', ' ', sentence)
    tokens = sentence.split('/')
    filtered_sentence = [w for w in tokens if not w in stop_words] 
    stemmed_tokens = [porter_stemmer.stem(token.strip()) for token in tokens if len(token) > 1]
    return ''.join(stemmed_tokens)

In [ ]:
filterd_crosstab_index['split'] = filterd_crosstab_index['referralPath'].apply(stem_sentences_v2)

In [ ]:
split_lst = filterd_crosstab_index['split'].values
split_lst

In [ ]:
lst_split_words = []
for i in split_lst:
    for j in i.split(' '):
        if len(j) > 2:
            lst_split_words.append(j)
lst_split_words_df = pd.DataFrame(lst_split_words, columns=['words'])
lst_split_words_df_unique = lst_split_words_df['words'].unique()
lst_split_words_df_unique

In [ ]:
def transform_referralPath(data):
    # given the data, want to make binary columns stating that they contain necessary keywords 
    lst_words_final = ['deal', 'sign', 'google', 'merchandise', 'store', 'emails',
                       'special', 'coup', 'stor', 'mail', 'com', 'forum', 'merch', 'new',
                       'url', 'site', 'mountain', 'view', 'php', 'offer', 'googletopia', 
                       'free', 'stuff', 'alphabet','discount']
    data_v1 = data
    data_v1['clean_referralPath'] = data_v1['referralPath'].apply(stem_sentences_v2)
    for i in lst_words_final:
        col_name = 'referralPath_' + str(i)
        data_v1[col_name] = data_v1['clean_referralPath'].str.contains(i, case=1, na=0)
        
    return data_v1

In [ ]:
%%time
train_data_v8 = transform_referralPath(train_data_v7)

### Focusing in on source

In [ ]:
pd.DataFrame(train_data_v8['source'].unique())

In [ ]:
crosstab = pd.crosstab(train_data_v8.source, train_data_v8.madePurchase)
crosstab = crosstab.sort_values(crosstab.columns[1], ascending = False)
crosstab[2] = round(crosstab[1]/crosstab[0],2)
crosstab = crosstab.sort_values(crosstab.columns[2], ascending = False)
crosstab

In [ ]:
filterd_crosstab = crosstab[(crosstab[1] > 5) & (crosstab[2] >= 0.01)]
filterd_crosstab_index = filterd_crosstab.index
filterd_crosstab

In [ ]:
filterd_crosstab_index = filterd_crosstab.index
filterd_crosstab_index

In [ ]:
def transform_source(data):
    # given the data, want to make binary columns stating that they contain necessary keywords 
    lst_words = ['mall.googleplex.com', 'dealspotr.com', 'mail.google.com',
       'groups.google.com', 'phandroid.com', 'gdeals.googleplex.com', 'dfa',
       'l.facebook.com', 'yahoo', 'google', 'bing', 'sites.google.com',
       '(direct)', 'facebook.com']
    data_v1 = data
    for i in lst_words:
        col_name = 'source_' + str(i)
        data_v1[col_name] = data_v1['source'].str.contains(i, case=1, na=0)
    return data_v1

In [ ]:
%%time
train_data_v9 = transform_source(train_data_v8)

In [ ]:
train_data_v9.shape

### Focusing in on region

In [ ]:
crosstab = pd.crosstab(train_data_v8.region, train_data_v8.madePurchase)
crosstab = crosstab.sort_values(crosstab.columns[1], ascending = False)
crosstab[2] = round(crosstab[1]/crosstab[0],2)
crosstab = crosstab.sort_values(crosstab.columns[2], ascending = False)
crosstab[:34]

In [ ]:
filtered_crosstab_index = crosstab[:34].index
filtered_crosstab_index

In [ ]:
def transform_region(data):
    # given the data, want to make binary columns stating that they contain necessary keywords 
    lst_words = ['Zulia', 'Nebraska', 'Michigan', 'Pichincha', 'Tennessee', 'Illinois',
       'New York', 'Washington', 'Massachusetts', 'Colorado', 'South Carolina',
       'Texas', 'Georgia', 'Missouri', 'Iowa', 'District of Columbia',
       'California', 'Minnesota', 'Utah', 'Arizona', 'Pennsylvania',
       'New Jersey', 'Indiana', 'Florida', 'Maryland', 'Connecticut',
       'North Carolina', 'Virginia', 'Nevada', 'Ohio', 'Alberta', '(not_set)',
       'Ontario', 'Oregon']
    data_v1 = data
    for i in lst_words:
        col_name = 'source_' + str(i)
        data_v1[col_name] = data_v1['source'].str.contains(i, case=1, na=0)
    return data_v1

In [ ]:
train_data_v10 = transform_region(train_data_v9)

### Focusing in on channelGrouping and medium

In [ ]:
train_data_v10['channelGrouping'].unique()

In [ ]:
train_data_v10['medium'].unique()

In [ ]:
test = train_data_v10[['medium', 'channelGrouping']]

In [ ]:
test["med_channel"] = test["medium"].map(str) + '_' + test["channelGrouping"].map(str)

In [ ]:
test['med_channel'].unique()

Decided to delete the medium column and just keep channel grouping

In [ ]:
train_data_v11 = train_data_v10.drop(columns = ['medium'])

### Focusing in on operatingSystem

In [ ]:
train_data_v11['operatingSystem'].unique()

In [ ]:
crosstab = pd.crosstab(train_data_v8.operatingSystem, train_data_v8.madePurchase)
crosstab = crosstab.sort_values(crosstab.columns[1], ascending = False)
crosstab[2] = round(crosstab[1]/crosstab[0],2)
crosstab = crosstab.sort_values(crosstab.columns[2], ascending = False)
crosstab

In [ ]:
filtered_crosstab =  crosstab[(crosstab[1] > 5) & (crosstab[2] >= 0.01)]
filtered_crosstab

In [ ]:
filtered_crosstab_index = filtered_crosstab.index.values
filtered_crosstab_index

In [ ]:
def transform_operatingSystem(data):
    # given the data, want to make binary columns stating that they contain necessary keywords 
    lst_words = ['Chrome OS', 'Macintosh', 'Linux', 'iOS', 'Windows']
    data_v1 = data
    for i in lst_words:
        col_name = 'operatingSystem_' + str(i)
        data_v1[col_name] = data_v1['operatingSystem'].str.contains(i, case=1, na=0)
    return data_v1

In [ ]:
train_data_v12 = transform_operatingSystem(train_data_v11)

### Focusing in on adContent

In [ ]:
train_data_v11['adContent'].unique()

In [ ]:
crosstab = pd.crosstab(train_data_v8.adContent, train_data_v8.madePurchase)
crosstab = crosstab.sort_values(crosstab.columns[1], ascending = False)
crosstab[2] = round(crosstab[1]/crosstab[0],2)
crosstab = crosstab.sort_values(crosstab.columns[2], ascending = False)
crosstab

In [ ]:
filtered_crosstab =  crosstab[(crosstab[1] > 5) & (crosstab[2] >= 0.01)]
filtered_crosstab

In [ ]:
filtered_crosstab_index = filtered_crosstab.index.values
filtered_crosstab_index

In [ ]:
def transform_adContent(data):
    # given the data, want to make binary columns stating that they contain necessary keywords 
    lst_words = ['Google Merchandise Collection', '(not_set)']
    data_v1 = data
    for i in lst_words:
        col_name = 'adContent' + str(i)
        data_v1[col_name] = data_v1['adContent'].str.contains(i, case=1, na=0)
    return data_v1

In [ ]:
train_data_v13 = transform_adContent(train_data_v12)

### Focusing in on Browser

In [ ]:
train_data_v13['browser'].unique()

In [ ]:
crosstab = pd.crosstab(train_data_v8.browser, train_data_v8.madePurchase)
crosstab = crosstab.sort_values(crosstab.columns[1], ascending = False)
crosstab[2] = round(crosstab[1]/crosstab[0],2)
crosstab = crosstab.sort_values(crosstab.columns[2], ascending = False)
crosstab

In [ ]:
filtered_crosstab =  crosstab[(crosstab[1] > 5) & (crosstab[2] >= 0.01)]
filtered_crosstab

In [ ]:
filtered_crosstab_index = filtered_crosstab.index.values
filtered_crosstab_index

In [ ]:
def transform_browser(data):
    # given the data, want to make binary columns stating that they contain necessary keywords 
    lst_words = ['Chrome', 'Firefox', 'Internet Explorer', 'Edge']
    data_v1 = data
    for i in lst_words:
        col_name = 'browser' + str(i)
        data_v1[col_name] = data_v1['browser'].str.contains(i, case=1, na=0)
    return data_v1

In [ ]:
train_data_v14 = transform_browser(train_data_v13)

In [ ]:
train_data_v14.head()

### Focusing in on metro

In [ ]:
train_data_v14['metro'].unique()

In [ ]:
crosstab = pd.crosstab(train_data_v8.metro, train_data_v8.madePurchase)
crosstab = crosstab.sort_values(crosstab.columns[1], ascending = False)
crosstab[2] = round(crosstab[1]/crosstab[0],2)
crosstab = crosstab.sort_values(crosstab.columns[2], ascending = False)
crosstab

Decided to use region instead of metro since it is cleaner!

In [ ]:
train_data_v15= train_data_v14.drop(columns=['metro'])

### Focusing in on country

In [ ]:
crosstab = pd.crosstab(train_data_v8.country, train_data_v8.madePurchase)
crosstab = crosstab.sort_values(crosstab.columns[1], ascending = False)
crosstab[2] = round(crosstab[1]/crosstab[0],2)
crosstab = crosstab.sort_values(crosstab.columns[2], ascending = False)
crosstab

In [ ]:
filtered_crosstab =  crosstab[(crosstab[1] > 5) & (crosstab[2] >= 0.01)]
filtered_crosstab

In [ ]:
filtered_crosstab_index = filtered_crosstab.index.values
filtered_crosstab_index

In [ ]:
def transform_country(data):
    # given the data, want to make binary columns stating that they contain necessary keywords 
    lst_words = ['United States', 'Venezuela', 'Puerto Rico', 'Canada']
    data_v1 = data
    for i in lst_words:
        col_name = 'country' + str(i)
        data_v1[col_name] = data_v1['country'].str.contains(i, case=1, na=0)
    return data_v1

In [ ]:
train_data_v16 = transform_country(train_data_v15)

### Focusing in on campaign

In [ ]:
crosstab = pd.crosstab(train_data_v8.campaign, train_data_v8.madePurchase)
crosstab = crosstab.sort_values(crosstab.columns[1], ascending = False)
crosstab[2] = round(crosstab[1]/crosstab[0],2)
crosstab = crosstab.sort_values(crosstab.columns[2], ascending = False)
crosstab

In [ ]:
filtered_crosstab =  crosstab[(crosstab[1] > 5) & (crosstab[2] >= 0.01)]
filtered_crosstab

In [ ]:
filtered_crosstab_index = filtered_crosstab.index.values
filtered_crosstab_index

In [ ]:
def transform_campaign(data):
    # given the data, want to make binary columns stating that they contain necessary keywords 
    lst_words = ['AW - Dynamic Search Ads Whole Site', 'AW - Accessories', '(not_set)']
    data_v1 = data
    for i in lst_words:
        col_name = 'campaign' + str(i)
        data_v1[col_name] = data_v1['campaign'].str.contains(i, case=1, na=0)
    return data_v1

In [ ]:
train_data_v17 = transform_campaign(train_data_v16)

### Focusing in on subContinent

In [ ]:
crosstab = pd.crosstab(train_data_v8.subContinent, train_data_v8.madePurchase)
crosstab = crosstab.sort_values(crosstab.columns[1], ascending = False)
crosstab[2] = round(crosstab[1]/crosstab[0],2)
crosstab = crosstab.sort_values(crosstab.columns[2], ascending = False)
crosstab

decided to delete the continent column and just keep one or two one hotebcoding columns from the subcontinent

In [ ]:
filtered_crosstab =  crosstab[(crosstab[1] > 5) & (crosstab[2] >= 0.01)]
filtered_crosstab

In [ ]:
filtered_crosstab_index = filtered_crosstab.index.values
filtered_crosstab_index

In [ ]:
def transform_subContinent(data):
    # given the data, want to make binary columns stating that they contain necessary keywords 
    lst_words = ['Northern America', 'Caribbean']
    data_v1 = data
    for i in lst_words:
        col_name = 'subContinent' + str(i)
        data_v1[col_name] = data_v1['subContinent'].str.contains(i, case=1, na=0)
    return data_v1

In [ ]:
train_data_v18 = transform_subContinent(train_data_v17)

### Focusing in on city

In [ ]:
train_data_v15['city'].unique()

In [ ]:
crosstab = pd.crosstab(train_data_v8.city, train_data_v8.madePurchase)
crosstab = crosstab.sort_values(crosstab.columns[1], ascending = False)
crosstab[2] = round(crosstab[1]/crosstab[0],2)
crosstab = crosstab.sort_values(crosstab.columns[2], ascending = False)
crosstab

In [ ]:
filtered_crosstab =  crosstab[(crosstab[1] > 5) & (crosstab[2] >= 0.01)]
filtered_crosstab

In [ ]:
filtered_crosstab_index = filtered_crosstab.index.values
filtered_crosstab_index

In [ ]:
def transform_city(data):
    # given the data, want to make binary columns stating that they contain necessary keywords 
    lst_words = ['Maracaibo', 'Ann Arbor', 'Cambridge', 'San Bruno', 'Chicago',
       'Austin', 'Irvine', 'New York', 'Nashville', 'Jersey City',
       'Boulder', 'Kirkland', 'Seattle', 'Oakland', 'Denver', 'Sunnyvale',
       'San Francisco', 'Pittsburgh', 'Washington', 'Atlanta',
       'Los Angeles', 'Mountain View', 'Minneapolis', 'San Antonio',
       'Lake Oswego', 'Santa Clara', 'Cupertino', 'Salem', 'San Mateo',
       'San Diego', 'Palo Alto', 'Fremont', 'Houston', 'Milpitas',
       'Boston', 'Charlotte', 'San Jose', 'Philadelphia', 'Redwood City',
       'Portland', 'Phoenix', '(not_set)', 'Toronto', 'Dallas']
    data_v1 = data
    for i in lst_words:
        col_name = 'city' + str(i)
        data_v1[col_name] = data_v1['city'].str.contains(i, case=1, na=0)
    return data_v1

In [ ]:
train_data_v19 = transform_city(train_data_v18)

In [ ]:
train_data_v19.head()

# Methods to Do all Data Cleaning and Feature Engineering

In [23]:
%%time
train_data = pd.read_csv('/Users/josephcolaco/customer_revenue_prediction/data/train.csv')
test_data = pd.read_csv('/Users/josephcolaco/customer_revenue_prediction/data/test.csv')
print('Shape of test data is', test_data.shape)
print('Shape of train data is',train_data.shape)
print(train_data['fullVisitorId'].nunique())

<string>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.


Shape of test data is (804684, 12)
Shape of train data is (903653, 12)
742735
CPU times: user 21.7 s, sys: 2.43 s, total: 24.1 s
Wall time: 24.3 s


In [4]:
def json_to_lst(model_data, json_cols):
    for column in json_cols:
        model_data = model_data.join(pd.DataFrame(
            model_data.pop(column).apply(pd.io.json.loads).values.tolist(), index=model_data.index)) 
    
    return model_data

In [35]:
def data_prep(data, is_train):
    # json columns to list
    json_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']
    data = json_to_lst(data, json_cols)
    # adwordsClickInfo still in json column
    concat_df = pd.DataFrame(data['adwordsClickInfo'].tolist())
    data = pd.concat([data,concat_df], axis=1)
    
    # dropping columns that have null values or provide no information
    data = data.drop(columns = ['adwordsClickInfo', 'browserSize', 'browserVersion', 'flashVersion', 'language',
                                'mobileDeviceBranding', 'mobileDeviceInfo', 'mobileDeviceMarketingName', 
                                'mobileDeviceModel', 'mobileInputSelector', 'operatingSystemVersion', 'screenColors',
                                'screenResolution', 'cityId', 'latitude', 'longitude', 'networkLocation', 
                                'targetingCriteria','criteriaParameters', 'visits', 'socialEngagementType', 
                                'isTrueDirect'])
    
    # adjusting data types
    data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')
    if is_train:
        data['transactionRevenue'] = data['transactionRevenue'].fillna(1).astype('float64')
    data['hits'] = data['hits'].fillna(0).astype('int64')
    data['newVisits'] = data['newVisits'].fillna(0).astype('int64')
    data['pageviews'] = data['pageviews'].fillna(0).astype('int64')
    data['page'] = data['page'].fillna(0).astype('int64')
    data['bounces'] = data['bounces'].fillna(0).astype('int64')
    data['isVideoAd'] = data['isVideoAd'].fillna('False').astype('int64')
    data['visitStartTime'] = data['visitStartTime'].apply(lambda x: dt.datetime.utcfromtimestamp(float(x)))
    data['visitHour'] = data['visitStartTime'].dt.hour
#     data['fullVisitorId'] = data['fullVisitorId'].astype('float64')
    
    # dealing with nulls
    for i in data.columns:
        if data[i].isnull().sum() > 0:
            data[i] = data[i].fillna('(not_set)')
            
    # dealing with string nulls      
    data = data.replace(['not available in demo dataset', '(not set)'], ['(not_set)', '(not_set)'])
    data['networkDomain'] = data['networkDomain'].replace(['unknown.unknown'], ['(not_set)'])
    data['referralPath'] = data['referralPath'].replace(['/'], ['(not_set)'])
    data['keyword'] = data['keyword'].replace(['(not provided)'], ['(not_set)'])
    
    if is_train:
        data = data.assign(
            logtransactionRevenue = np.log(data.transactionRevenue))
        data['madePurchase'] = np.where(data['logtransactionRevenue'] > 0, 1, 0)
    
    print(data['fullVisitorId'].nunique(), ' 1')
    # dealing with duplicate unique identifiers
    _, data =  remove_session_id_dup(data, is_train)
    print(data['fullVisitorId'].nunique(), ' 2')
    if is_train:
        data = data.drop(columns=['transactionRevenue', 'index', 'visitId', 'visitStartTime', 'gclId'])
    else:
        data = data.drop(columns=['index', 'visitId', 'visitStartTime', 'gclId'])
        
    return data

In [33]:
def remove_session_id_dup(data, is_train):
    sessionId_counts = pd.DataFrame(data['sessionId'].value_counts()).reset_index()
    dup_sessionId = sessionId_counts['index'][sessionId_counts['sessionId']>1].values.tolist()
    dup_data = data[data['sessionId'].isin(dup_sessionId)].sort_values(by='sessionId')
    
    index_to_delete = []
    count = 0
    delete = False
    for index, row in dup_data.iterrows():
        if is_train:
            if count == 0 and row['madePurchase'] == 1: #leaves the first duplicate if the person has made a purchase
                count +=1
                delete = False
                continue
            elif count == 0 and delete == False: #deletes the first duplicate if person hasnt made a purchase
                index_to_delete.append(index)
                delete = True
                count +=1
            elif count == 1 and delete == False: #deletes the second duplicate if person has made a purchas in the first row
                index_to_delete.append(index)
                count = 0
            elif count == 1 and delete == True: #assign delete to false and continues
                delete = False
                count = 0
                continue
        elif count == 0 and delete == False:
            index_to_delete.append(index)
            delete = True
            count +=1
        else:
            delete = False
            count = 0
            
    data = data.reset_index()
    print('Deleted', len(index_to_delete), 'rows!')
    print('Shape of data before deleting duplicates:', data.shape)
    data_v1 = data[~data['index'].isin(index_to_delete)]
    print('Shape of data after deleting duplicates:', data_v1.shape)   
    
    return index_to_delete, data_v1

In [ ]:
%%time
print(train_data.shape)
train_data_v1 = data_prep(train_data, is_train=True)
print(train_data_v1.shape)

In [9]:
train_data['fullVisitorId'].nunique()

742735

In [8]:
train_data_v1['fullVisitorId'].nunique()

742532

In [34]:
%%time
print(test_data.shape)
test_data_v1 = data_prep(test_data, is_train=False)
print(test_data_v1.shape)

(804684, 11)


KeyError: 'device'

In [27]:
test_data['fullVisitorId'].nunique()

652237

In [28]:
test_data_v1['fullVisitorId'].nunique()

652040

In [10]:
def stem_sentences(sentence):
    #     nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    porter_stemmer = PorterStemmer()
    sentence = re.sub('[^a-zA-Z]+', ' ', sentence)
    tokens = sentence.split()
    filtered_sentence = [w for w in tokens if not w in stop_words] 
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens if len(token) > 1]
    return ' '.join(stemmed_tokens)

In [11]:
def stem_sentences_v2(sentence):
    stop_words = set(stopwords.words('english'))
    porter_stemmer = PorterStemmer()
    sentence = re.sub('[^a-zA-Z]+', ' ', sentence)
    tokens = sentence.split('/')
    filtered_sentence = [w for w in tokens if not w in stop_words] 
    stemmed_tokens = [porter_stemmer.stem(token.strip()) for token in tokens if len(token) > 1]
    return ''.join(stemmed_tokens)

In [12]:
def make_columns(data, data_col_name, col_list, col_mark):
    
    for i in col_list:
        col_name = col_mark + str(i)
        data[col_name] = data[data_col_name].str.contains(i, case=1, na=0)
        data[col_name] = data[col_name].astype(int)
        
    data = data.drop(columns=[data_col_name])
    return data

In [13]:
def feature_eng(data):
    # making features out of the date column
    data['dayNameDate'] = data['date'].dt.day_name()
    data['monthDate'] = data['date'].dt.month
    data['yearDate'] = data['date'].dt.year
    data['dayDate'] = data['date'].dt.day
    
    # make columns
    network_domain_lst = ['(not_set)', '.us', '.net', '.com', '.edu', '.ca', '.org', '.mx' ]
    operating_system_lst = ['Chrome OS', 'Macintosh', 'Linux', 'iOS', 'Windows']
    ad_content_lst = ['Google Merchandise Collection', '(not_set)']
    browser_lst = ['Chrome', 'Firefox', 'Internet Explorer', 'Edge']
    country_lst = ['United States', 'Venezuela', 'Puerto Rico', 'Canada']
    subContinent_lst = ['Northern America', 'Caribbean']
    campaign_lst = ['AW - Dynamic Search Ads Whole Site', 'AW - Accessories', '(not_set)']
    region_lst = ['Zulia', 'Nebraska', 'Michigan', 'Pichincha', 'Tennessee', 'Illinois',
       'New York', 'Washington', 'Massachusetts', 'Colorado', 'South Carolina',
       'Texas', 'Georgia', 'Missouri', 'Iowa', 'District of Columbia',
       'California', 'Minnesota', 'Utah', 'Arizona', 'Pennsylvania',
       'New Jersey', 'Indiana', 'Florida', 'Maryland', 'Connecticut',
       'North Carolina', 'Virginia', 'Nevada', 'Ohio', 'Alberta', '(not_set)',
       'Ontario', 'Oregon']
    source_lst = ['mall.googleplex.com', 'dealspotr.com', 'mail.google.com',
       'groups.google.com', 'phandroid.com', 'gdeals.googleplex.com', 'dfa',
       'l.facebook.com', 'yahoo', 'google', 'bing', 'sites.google.com',
       '(direct)', 'facebook.com']
    city_lst = ['Maracaibo', 'Ann Arbor', 'Cambridge', 'San Bruno', 'Chicago',
       'Austin', 'Irvine', 'New York', 'Nashville', 'Jersey City',
       'Boulder', 'Kirkland', 'Seattle', 'Oakland', 'Denver', 'Sunnyvale',
       'San Francisco', 'Pittsburgh', 'Washington', 'Atlanta',
       'Los Angeles', 'Mountain View', 'Minneapolis', 'San Antonio',
       'Lake Oswego', 'Santa Clara', 'Cupertino', 'Salem', 'San Mateo',
       'San Diego', 'Palo Alto', 'Fremont', 'Houston', 'Milpitas',
       'Boston', 'Charlotte', 'San Jose', 'Philadelphia', 'Redwood City',
       'Portland', 'Phoenix', '(not_set)', 'Toronto', 'Dallas']
    referalPath_lst = ['deal', 'sign', 'google', 'merchandise', 'store', 'emails',
                       'special', 'coup', 'stor', 'mail', 'com', 'forum', 'merch', 'new',
                       'url', 'site', 'mountain', 'view', 'php', 'offer', 'googletopia', 
                       'free', 'stuff', 'alphabet','discount']
    keyword_lst = ['qehscssdk', 'googl', 'merchandis', 'store', 'hzbaqlcbjwfgoh', 
                       'remarket', 'content', 'target', 'zknv']
    
    data = make_columns(data, 'networkDomain', network_domain_lst, 'domain_')
    data = make_columns(data, 'operatingSystem', operating_system_lst, 'os_')
    data = make_columns(data, 'adContent', ad_content_lst, 'adContent_')
    data = make_columns(data, 'browser', browser_lst, 'browser_')
    data = make_columns(data, 'country', country_lst, 'country_')
    data = make_columns(data, 'city', city_lst, 'city_')
    data = make_columns(data, 'subContinent', subContinent_lst, 'subContinent_')
    data = make_columns(data, 'campaign', campaign_lst, 'campaign_')
    data = make_columns(data, 'region', region_lst, 'region_')
    data = make_columns(data, 'source', source_lst, 'source_')
    

    data['clean_referralPath'] = data['referralPath'].apply(stem_sentences_v2)
    data = make_columns(data, 'clean_referralPath', referalPath_lst, 'referralPath_')
    
    data['clean_keyword'] = data['keyword'].apply(stem_sentences)
    data = make_columns(data, 'clean_keyword', keyword_lst, 'keyword_')
    
    data = data.drop(columns = ['date', 'metro', 'medium', 'referralPath', 'keyword'])
    return data

In [14]:
%%time
train_data_v2 = feature_eng(train_data_v1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


CPU times: user 8min 49s, sys: 1min 5s, total: 9min 54s
Wall time: 9min 20s


In [52]:
%%time
test_data_v2 = feature_eng(test_data_v1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


CPU times: user 8min 14s, sys: 1min 4s, total: 9min 18s
Wall time: 8min 56s


In [54]:
test_data_v2.shape

(803863, 174)

In [56]:
test_data_v2.head()

,channelGrouping,fullVisitorId,sessionId,visitNumber,deviceCategory,isMobile,continent,bounces,hits,newVisits,pageviews,adNetworkType,isVideoAd,page,slot,visitHour,dayNameDate,monthDate,yearDate,dayDate,domain_(not_set),domain_.us,domain_.net,domain_.com,domain_.edu,domain_.ca,domain_.org,domain_.mx,os_Chrome OS,os_Macintosh,os_Linux,os_iOS,os_Windows,adContent_Google Merchandise Collection,adContent_(not_set),browser_Chrome,browser_Firefox,browser_Internet Explorer,browser_Edge,country_United States,country_Venezuela,country_Puerto Rico,country_Canada,city_Maracaibo,city_Ann Arbor,city_Cambridge,city_San Bruno,city_Chicago,city_Austin,city_Irvine,city_New York,city_Nashville,city_Jersey City,city_Boulder,city_Kirkland,city_Seattle,city_Oakland,city_Denver,city_Sunnyvale,city_San Francisco,city_Pittsburgh,city_Washington,city_Atlanta,city_Los Angeles,city_Mountain View,city_Minneapolis,city_San Antonio,city_Lake Oswego,city_Santa Clara,city_Cupertino,city_Salem,city_San Mateo,city_San Diego,city_Palo Alto,city_Fremont,city_Houston,city_Milpitas,city_Boston,city_Charlotte,city_San Jose,city_Philadelphia,city_Redwood City,city_Portland,city_Phoenix,city_(not_set),city_Toronto,city_Dallas,subContinent_Northern America,subContinent_Caribbean,campaign_AW - Dynamic Search Ads Whole Site,campaign_AW - Accessories,campaign_(not_set),region_Zulia,region_Nebraska,region_Michigan,region_Pichincha,region_Tennessee,region_Illinois,region_New York,region_Washington,region_Massachusetts,region_Colorado,region_South Carolina,region_Texas,region_Georgia,region_Missouri,region_Iowa,region_District of Columbia,region_California,region_Minnesota,region_Utah,region_Arizona,region_Pennsylvania,region_New Jersey,region_Indiana,region_Florida,region_Maryland,region_Connecticut,region_North Carolina,region_Virginia,region_Nevada,region_Ohio,region_Alberta,region_(not_set),region_Ontario,region_Oregon,source_mall.googleplex.com,source_dealspotr.com,source_mail.google.com,source_groups.google.com,source_phandroid.com,source_gdeals.googleplex.com,source_dfa,source_l.facebook.com,source_yahoo,source_google,source_bing,source_sites.google.com,source_(direct),source_facebook.com,referralPath_deal,referralPath_sign,referralPath_google,referralPath_merchandise,referralPath_store,referralPath_emails,referralPath_special,referralPath_coup,referralPath_stor,referralPath_mail,referralPath_com,referralPath_forum,referralPath_merch,referralPath_new,referralPath_url,referralPath_site,referralPath_mountain,referralPath_view,referralPath_php,referralPath_offer,referralPath_googletopia,referralPath_free,referralPath_stuff,referralPath_alphabet,referralPath_discount,keyword_qehscssdk,keyword_googl,keyword_merchandis,keyword_store,keyword_hzbaqlcbjwfgoh,keyword_remarket,keyword_content,keyword_target,keyword_zknv
0,Organic Search,6.167871e+18,6167871330617112363_1508151024,2,desktop,False,Asia,0,4,0,4,(not_set),True,0,(not_set),10,Monday,10,2017,16,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Organic Search,6.436976e+17,0643697640977915618_1508175522,1,desktop,False,Europe,0,5,1,5,(not_set),True,0,(not_set),17,Monday,10,2017,16,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Organic Search,6.059384e+18,6059383810968229466_1508143220,1,desktop,False,Europe,0,7,1,7,(not_set),True,0,(not_set),8,Monday,10,2017,16,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0

In [55]:
test_data_v2.to_csv('~/customer_revenue_prediction/data/cleaned_feat_eng_test_data.csv')

In [59]:
train_data_v2.head()

,channelGrouping,fullVisitorId,sessionId,visitNumber,deviceCategory,isMobile,continent,bounces,hits,newVisits,pageviews,campaignCode,adNetworkType,isVideoAd,page,slot,visitHour,logtransactionRevenue,madePurchase,dayNameDate,monthDate,yearDate,dayDate,domain_(not_set),domain_.us,domain_.net,domain_.com,domain_.edu,domain_.ca,domain_.org,domain_.mx,os_Chrome OS,os_Macintosh,os_Linux,os_iOS,os_Windows,adContent_Google Merchandise Collection,adContent_(not_set),browser_Chrome,browser_Firefox,browser_Internet Explorer,browser_Edge,country_United States,country_Venezuela,country_Puerto Rico,country_Canada,city_Maracaibo,city_Ann Arbor,city_Cambridge,city_San Bruno,city_Chicago,city_Austin,city_Irvine,city_New York,city_Nashville,city_Jersey City,city_Boulder,city_Kirkland,city_Seattle,city_Oakland,city_Denver,city_Sunnyvale,city_San Francisco,city_Pittsburgh,city_Washington,city_Atlanta,city_Los Angeles,city_Mountain View,city_Minneapolis,city_San Antonio,city_Lake Oswego,city_Santa Clara,city_Cupertino,city_Salem,city_San Mateo,city_San Diego,city_Palo Alto,city_Fremont,city_Houston,city_Milpitas,city_Boston,city_Charlotte,city_San Jose,city_Philadelphia,city_Redwood City,city_Portland,city_Phoenix,city_(not_set),city_Toronto,city_Dallas,subContinent_Northern America,subContinent_Caribbean,campaign_AW - Dynamic Search Ads Whole Site,campaign_AW - Accessories,campaign_(not_set),region_Zulia,region_Nebraska,region_Michigan,region_Pichincha,region_Tennessee,region_Illinois,region_New York,region_Washington,region_Massachusetts,region_Colorado,region_South Carolina,region_Texas,region_Georgia,region_Missouri,region_Iowa,region_District of Columbia,region_California,region_Minnesota,region_Utah,region_Arizona,region_Pennsylvania,region_New Jersey,region_Indiana,region_Florida,region_Maryland,region_Connecticut,region_North Carolina,region_Virginia,region_Nevada,region_Ohio,region_Alberta,region_(not_set),region_Ontario,region_Oregon,source_mall.googleplex.com,source_dealspotr.com,source_mail.google.com,source_groups.google.com,source_phandroid.com,source_gdeals.googleplex.com,source_dfa,source_l.facebook.com,source_yahoo,source_google,source_bing,source_sites.google.com,source_(direct),source_facebook.com,referralPath_deal,referralPath_sign,referralPath_google,referralPath_merchandise,referralPath_store,referralPath_emails,referralPath_special,referralPath_coup,referralPath_stor,referralPath_mail,referralPath_com,referralPath_forum,referralPath_merch,referralPath_new,referralPath_url,referralPath_site,referralPath_mountain,referralPath_view,referralPath_php,referralPath_offer,referralPath_googletopia,referralPath_free,referralPath_stuff,referralPath_alphabet,referralPath_discount,keyword_qehscssdk,keyword_googl,keyword_merchandis,keyword_store,keyword_hzbaqlcbjwfgoh,keyword_remarket,keyword_content,keyword_target,keyword_zknv
0,Organic Search,1.131660e+18,1131660440785968503_1472830385,1,desktop,False,Asia,1,1,1,1,(not_set),(not_set),True,0,(not_set),15,0.0,0,Friday,9,2016,2,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Organic Search,3.773060e+17,377306020877927890_1472880147,1,desktop,False,Oceania,1,1,1,1,(not_set),(not_set),True,0,(not_set),5,0.0,0,Friday,9,2016,2,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Organic Search,3.895546e+18,3895546263509774583_1472865386,1,desktop,False,Europe,1,1,1,1,(not_set),(not_set),True,0,(not_set),1,0.0,0,Friday,9,2016,2,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
train_data_v2.shape

In [16]:
train_data_v2.to_csv('~/customer_revenue_prediction/data/cleaned_feat_eng_train_data.csv')

# Model Implementation

### Idea 1
    - Run logistic regression for classification with a select few features
    - Run logistic regression to predict logtransactionrevenue for the rows that were classified as 1

### Logistic Regression Evaluation Metrics for Classification Model
    - confusion matrix
    - ROC curve
    - accuracy
    
Since this is the first iteration, no kfold
https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8

In [ ]:
train_data_v2.dtypes

In [ ]:
y = train_data_v2['madePurchase']
train_data_v3 = train_data_v2.drop(columns = ['sessionId', 'logtransactionRevenue', 'madePurchase', 'keyword'])

In [ ]:
%%time
# train_v2 = pd.get_dummies(train_v1)
train_data_model = pd.get_dummies(train_data_v3)

In [ ]:
print(train_data_model.shape)
print(y.shape)

In [ ]:
%%time
X_train, X_validate, y_train, y_validate = train_test_split(train_data_model, y, test_size=0.2, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_validate)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_validate, y_validate)))

In [ ]:
print(classification_report(y_validate, y_pred))

In [ ]:
logit_roc_auc = roc_auc_score(y_validate, logreg.predict(X_validate))
fpr, tpr, thresholds = roc_curve(y_validate, logreg.predict_proba(X_validate)[:,1])
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.legend(loc="lower right")

### Try SMOTE to Even Out Samples

In [ ]:
# sample a 100000 rows
subset_train_data_v3 = train_data_v3.sample(100000, random_state=0)
subset_y = y.sample(100000, random_state=0)

In [ ]:
subset_y.value_counts()

In [ ]:
# X = subset_train_data_v5[['channelGrouping', 'visitNumber', 'deviceCategory', 'isMobile',
#                           'continent', 'bounces', 'hits','newVisits', 'pageviews', 'adContent', 'isVideoAd',
#                           'page', 'slot', 'visitHour', 'dayNameDate', 'monthDate']]
X_v1 = pd.get_dummies(subset_train_data_v3)

In [ ]:
os = SMOTE(random_state=0)
X_train, X_validate, y_train, y_validate = train_test_split(X_v1, subset_y, test_size=0.2, random_state=0)
columns = X_train.columns

In [ ]:
%%time
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])

In [ ]:
print('This is the original number of rows of the subset sample:', subset_y.shape[0])
print('This is the new number of rows of the subset sample:', os_data_y.shape[0])
print('Number of rows that have made a purchase increased from',
      subset_y[subset_y==1].shape[0],'to',os_data_y[os_data_y.y == 1].shape[0] )

In [ ]:
os_data_X.shape

In [ ]:
%%time
logreg = LogisticRegression()
rfe = RFE(logreg, 50)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
col_bool = rfe.support_
rfe_cols = []
for i in enumerate(os_data_X.columns):
    if col_bool[i[0]] == True:
        rfe_cols.append(i[1])
os_data_X_rfe = os_data_X[rfe_cols]

In [ ]:
os_data_X_rfe.shape

In [ ]:
os_data_X_rfe.head()

In [ ]:
%%time
logit_model=sm.Logit(os_data_y,os_data_X_rfe)
result=logit_model.fit()
print(result.summary())

In [ ]:
%%time
X_train, X_validate, y_train, y_validate = train_test_split(os_data_X_rfe, os_data_y, test_size=0.2, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_validate)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_validate, y_validate)))

In [ ]:
print(classification_report(y_validate, y_pred))

In [ ]:
logit_roc_auc = roc_auc_score(y_validate, logreg.predict(X_validate))
fpr, tpr, thresholds = roc_curve(y_validate, logreg.predict_proba(X_validate)[:,1])
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.legend(loc="lower right")

In [ ]:
os_data_X_rfe.head()

### Apply PCA on RFE Data and Plot to Identify Clusters

In [ ]:
scaler = MinMaxScaler()
scaled_os_data_X_rfe = scaler.fit_transform(os_data_X_rfe)

In [ ]:
scaled_os_data_X_rfe = pd.DataFrame(scaled_os_data_X_rfe)

In [ ]:
pca = PCA(n_components=2)
pca_scaled_os_data_X_rfe = pca.fit_transform(scaled_os_data_X_rfe)

In [ ]:
pca_scaled_os_data_X_rfe = pd.DataFrame(pca_scaled_os_data_X_rfe)

In [ ]:
sns_plot_data = pd.concat([pca_scaled_os_data_X_rfe,os_data_y ], axis=1)

In [ ]:
sns_plot_data.columns = ['x1', 'x2', 'y']

In [ ]:
sns.lmplot('x1', 'x2', data=sns_plot_data, hue='y', scatter_kws={'alpha':0.3}, fit_reg=False)

In [ ]:
sns.lmplot('x1', 'x2', data=sns_plot_data, scatter_kws={'alpha':0.3}, fit_reg=False)

You can clearly see from the data that there is a difference between 1 and 0 for made purchases!

Threshold for feature eng method for certain columns
    - max percentage of dataset
    - column name
    - list of words to make columns
    - stemming of columns
        - if stemming, what to split the word by?

For week of Sep 24th
    - See how the logistic method implemented above performs when certain rows for the browsers are deleted. If it performs the same or better move that into the feature engineering method because data is not too much of an issue
    - Run logistic regression models on sampled data with various RFE components to see which is the best starting with all

- build a simple model
    - Build a model to classify data as made transaction or did not make transaction
        - optimize that model and identify if possible to move on to next stage and predict transaction revenue
    - Build model to sum transaction for the specific row using the training data of ones with only transactions
        - identify if this is a better approach then using all models
       

Models that I will test out:
    - Logistic Regression
    - Random Forrest
    - Boosting
    - SVM